In [ ]:
"""
Created on 20190316

Learn pso and its implementation

Ref: https://github.com/DEAP/deap/blob/master/examples/pso/basic.py
"""

## Import

In [1]:
import operator
import random

import numpy

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

## Representation

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)

## Operators

### Initializer

In [27]:
def generate(size, pmin, pmax, smin, smax):
    # randomly generate the initial values
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size))
    # speed for each one
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

### Update
`The function updateParticle() first computes the speed, then limits the speed values between smin and smax, and finally computes the new particle position.`

In [36]:
def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))  # p_best - xi
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))  # g_best - xi
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))

In [ ]:
##

In [ ]:
def h1(individual):
    """ Simple two-dimensional function containing several local maxima.
    From: The Merits of a Parallel Genetic Algorithm in Solving Hard 
    Optimization Problems, A. J. Knoek van Soest and L. J. R. Richard 
    Casius, J. Biomech. Eng. 125, 141 (2003)

    .. list-table:: 
       :widths: 10 50
       :stub-columns: 1

       * - Type
         - maximization
       * - Range
         - :math:`x_i \in [-100, 100]`
       * - Global optima
         - :math:`\mathbf{x} = (8.6998, 6.7665)`, :math:`f(\mathbf{x}) = 2`\n
       * - Function
         - :math:`f(\mathbf{x}) = \\frac{\sin(x_1 - \\frac{x_2}{8})^2 + \
            \\sin(x_2 + \\frac{x_1}{8})^2}{\\sqrt{(x_1 - 8.6998)^2 + \
            (x_2 - 6.7665)^2} + 1}`

    .. plot:: code/benchmarks/h1.py
       :width: 67 %
    """
    num = (sin(individual[0] - individual[1] / 8))**2 + (sin(individual[1] + individual[0] / 8))**2
    denum = ((individual[0] - 8.6998)**2 + (individual[1] - 6.7665)**2)**0.5 + 1
    return num / denum,

In [37]:
toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

## Algorithm

In [39]:
def main():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best

if __name__ == "__main__":
    _, _, best = main()

gen	evals	avg      	std      	min      	max     
0  	5    	0.0903401	0.0607666	0.0266594	0.196602
1  	5    	0.12154  	0.0463369	0.0670736	0.188513
2  	5    	0.102484 	0.0526806	0.00301934	0.150215
3  	5    	0.125096 	0.0566785	0.0418374 	0.192794
4  	5    	0.122314 	0.0136061	0.110376  	0.146321
5  	5    	0.107978 	0.0311247	0.0779156 	0.165388
6  	5    	0.146148 	0.0483057	0.0634638 	0.196737
7  	5    	0.123523 	0.0816059	0.00993132	0.211463
8  	5    	0.188362 	0.170497 	0.0183077 	0.498348
9  	5    	0.383251 	0.300743 	0.121959  	0.925659
10 	5    	0.358693 	0.312665 	0.0700993 	0.906586
11 	5    	0.349155 	0.122885 	0.152391  	0.494697
12 	5    	0.309699 	0.130946 	0.115387  	0.51109 
13 	5    	0.377041 	0.19275  	0.168765  	0.705308
14 	5    	0.478703 	0.517999 	0.0644247 	1.49716 
15 	5    	0.409465 	0.0430045	0.350666  	0.461309
16 	5    	0.334686 	0.173306 	0.00815312	0.5168  
17 	5    	0.2666   	0.148494 	0.122105  	0.498518
18 	5    	0.313768 	0.31023  	0.00986911	0.763363
19 

514	5    	0.208306 	0.110015 	0.0279818  	0.364603
515	5    	0.430837 	0.431245 	0.0110786  	1.2571  
516	5    	0.466873 	0.178266 	0.204832   	0.733767
517	5    	0.407327 	0.0746074	0.341051   	0.55278 
518	5    	0.254031 	0.166673 	0.000933717	0.469406
519	5    	0.521059 	0.383833 	0.211329   	1.26371 
520	5    	0.312505 	0.139012 	0.0747227  	0.444807
521	5    	0.220874 	0.120928 	0.0354691  	0.415214
522	5    	0.495425 	0.182065 	0.311503   	0.842201
523	5    	0.271749 	0.125078 	0.0427481  	0.386873
524	5    	0.52472  	0.348707 	0.215076   	1.20055 
525	5    	0.394423 	0.116523 	0.274908   	0.605065
526	5    	0.613134 	0.350647 	0.343547   	1.30672 
527	5    	0.417436 	0.118298 	0.302301   	0.643248
528	5    	0.563846 	0.46544  	0.224872   	1.48055 
529	5    	0.237138 	0.0806649	0.111701   	0.3329  
530	5    	0.423512 	0.542395 	0.0984784  	1.50538 
531	5    	0.310555 	0.27459  	0.0736934  	0.83506 
532	5    	0.336305 	0.484265 	0.0165193  	1.29098 
533	5    	0.215989 	0.102072 	0

890	5    	0.374172 	0.123389 	0.201067   	0.561161
891	5    	0.269192 	0.101196 	0.0914891  	0.381095
892	5    	0.439816 	0.409555 	0.0886364  	1.20888 
893	5    	0.409571 	0.342626 	0.0348317  	1.03704 
894	5    	0.480051 	0.522649 	0.0471516  	1.49485 
895	5    	0.49522  	0.176252 	0.229617   	0.74043 
896	5    	0.494403 	0.376912 	0.0274038  	1.17605 
897	5    	0.335876 	0.141112 	0.140723   	0.573063
898	5    	0.247386 	0.147151 	0.0259398  	0.450118
899	5    	0.889732 	0.473466 	0.139676   	1.43921 
900	5    	0.183377 	0.124293 	0.0131616  	0.338829
901	5    	0.223004 	0.151407 	0.0534633  	0.473714
902	5    	0.328399 	0.368452 	0.0223154  	0.98429 
903	5    	0.304501 	0.217732 	0.00384142 	0.550462
904	5    	0.177011 	0.130644 	0.0393595  	0.396904
905	5    	0.203468 	0.13034  	0.0189559  	0.420736
906	5    	0.557177 	0.248571 	0.277538   	0.931975
907	5    	0.134914 	0.0921256	0.00719679 	0.239413
908	5    	0.354703 	0.17493  	0.10083    	0.646328
909	5    	0.487871 	0.352218 	0

In [40]:
best

[8.71981912931389, 6.788147249242675]